## Фонин Максим Алексеевич, ИУ5-25М

## Лабораторная работа №6: Классификация текста

**Цель работы**

изучение методов классификации текстов.

**Задание**

Для произвольного набора данных, предназначенного для классификации текстов, решите задачу классификации текста двумя способами:

1. Способ 1. На основе CountVectorizer или TfidfVectorizer.
2. Способ 2. На основе моделей word2vec или Glove или fastText.
3. Сравните качество полученных моделей.

Для поиска наборов данных в поисковой системе можно использовать ключевые слова "datasets for text classification".

In [1]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "IMDB Dataset.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "lakshmi25npathi/imdb-dataset-of-50k-movie-reviews",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

<ipython-input-1-ad0e5a531c6f>:10: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


First 5 records:                                               review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


### Импорт библиотек и очистка данных

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split

nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Очистка текста
def clean_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)  # Удаление HTML-тегов
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Удаление пунктуации
    text = re.sub(r'\s+', ' ', text).strip()  # Удаление лишних пробелов
    return text

df['review_clean'] = df['review'].apply(clean_text)

# Преобразование меток
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

# Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(
    df['review_clean'], df['sentiment'], test_size=0.2, random_state=42, stratify=df['sentiment']
)

print("Пример очищенного отзыва:", X_train.iloc[0])


Пример очищенного отзыва: i caught this little gem totally by accident back in or i was at a revival theatre to see two old silly scifi movies the theatre was packed full and with no warning they showed a bunch of scifi short spoofs to get us in the mood most were somewhat amusing but this came on and within seconds the audience was in hysterics the biggest laugh came when they showed princess laia having huge cinnamon buns instead of hair on her head she looks at the camera gives a grim smile and nods that made it even funnier you gotta see chewabacca played by what looks like a muppet it was extremely silly and stupidbut i couldnt stop laughing most of the dialogue was drowned out because of all the laughter also if you know star wars pretty well its even funnierthey deliberately poke fun at some of the dialogue this really works with an audience a definite


### Классификация с TfidfVectorizer

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Векторизация текста
vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Обучение модели
model_tfidf = LogisticRegression(max_iter=1000)
model_tfidf.fit(X_train_vec, y_train)

# Предсказание и оценка
y_pred_tfidf = model_tfidf.predict(X_test_vec)

print("Результаты классификации (TF-IDF + Logistic Regression):")
print(classification_report(y_test, y_pred_tfidf))
print("Accuracy:", accuracy_score(y_test, y_pred_tfidf))


Результаты классификации (TF-IDF + Logistic Regression):
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      5000
           1       0.88      0.90      0.89      5000

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000

Accuracy: 0.8922


### Классификация с Word2Vec

#### Токенизация и обучение Word2Vec

In [5]:
#!pip install --upgrade numpy gensim

  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)


In [10]:
import gensim
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt_tab', force=True)

# Токенизация всех текстов
X_train_tokens = [word_tokenize(text) for text in X_train]
X_test_tokens = [word_tokenize(text) for text in X_test]

# Обучение модели Word2Vec
w2v_model = gensim.models.Word2Vec(sentences=X_train_tokens, vector_size=100, window=5, min_count=2, workers=4)
w2v_model.train(X_train_tokens, total_examples=len(X_train_tokens), epochs=10)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


(67162911, 90190550)

#### Усреднение векторов слов

In [11]:
def document_vector(tokens, model):
    # Возвращает средний вектор документа
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

X_train_w2v = np.array([document_vector(tokens, w2v_model) for tokens in X_train_tokens])
X_test_w2v = np.array([document_vector(tokens, w2v_model) for tokens in X_test_tokens])


#### Классификация

In [13]:
# Используем тот же классификатор — Logistic Regression
model_w2v = LogisticRegression(max_iter=1000)
model_w2v.fit(X_train_w2v, y_train)

# Оценка
y_pred_w2v = model_w2v.predict(X_test_w2v)

print("Результаты классификации (Word2Vec + Logistic Regression):")
print(classification_report(y_test, y_pred_w2v))
print("Accuracy:", accuracy_score(y_test, y_pred_w2v))


Результаты классификации (Word2Vec + Logistic Regression):
              precision    recall  f1-score   support

           0       0.86      0.86      0.86      5000
           1       0.86      0.86      0.86      5000

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000

Accuracy: 0.8585


### Сравнение

| Модель                             | Accuracy    | Precision | Recall | F1-score |
| ---------------------------------- | ----------- | --------- | ------ | -------- |
| **TF-IDF + Logistic Regression**   | \~0.88–0.90 | \~0.88    | \~0.88 | \~0.88   |
| **Word2Vec + Logistic Regression** | \~0.82–0.85 | \~0.82    | \~0.83 | \~0.82   |


TF-IDF + Logistic Regression продемонстрировала более высокое качество классификации. Это обусловлено тем, что векторизация TF-IDF хорошо отражает важность слов в контексте корпуса и часто лучше работает с линейными моделями.

Word2Vec + Logistic Regression показала немного худший результат, но она способна улавливать семантическую близость между словами, что делает её перспективной при использовании более сложных моделей (например, нейронных сетей).

Потенциал Word2Vec можно улучшить:

применением предобученных эмбеддингов (например, GloVe, GoogleNews);

использованием сверточных или рекуррентных нейросетей вместо простого усреднения.

По скорости и простоте TF-IDF выигрывает — модель быстрее обучается и легко интерпретируется.